In [20]:
#files from this example downloaded from NASA MoonTrek
#link below has layers enabled for download
#https://trek.nasa.gov/moon/#v=0.1&x=-2.3071288632138396&y=-12.604064706295468&z=9&p=urn%3Aogc%3Adef%3Acrs%3AEPSG%3A%3A104903&d=&l=LRO_NAC_DEM_13S358E_200cmp%2Ctrue%2C1&l=LRO_NAC_Mosaic_13S358E_50cmp%2Ctrue%2C1&locale=&b=moon&e=-3.563690119071365%2C-13.404693353470263%2C-1.0505676073563142%2C-11.803436059120672&sfz=&w=

#enter filepaths
image_filepath = "C:/Users/Micah Acinapura/Downloads/LRO_NAC_Mosaic_13S358E_50cmp.tif"
height_filepath = "C:/Users/Micah Acinapura/Downloads/LRO_NAC_DEM_13S358E_200cmp.tif"
#enter name, description
layer_name = "Alphonsus"
layer_description = "Alphonsus LROC image and dem from MoonTrek"

#output folder for converted .tif, .vrt, and .info files
output_path = "C:/os/OpenSpaceData/Moon2/"
globe_for_layers = 'Moon'

print("Variables set")

Variables set


In [21]:
#requires osgeo/gdal.
#I installed with $conda install -c conda-forge gdal
from osgeo import gdal

#warp tifs for openspace projection
intput_image_data = gdal.Open(image_filepath);
srcband = intput_image_data.GetRasterBand(1);
image_nodata = srcband.GetNoDataValue()
print('warping image to long lat projection...please wait...')
output_image_filepath = output_path + layer_name + '.tif'
output_image_data = gdal.Warp(output_image_filepath, intput_image_data, dstSRS='+proj=longlat')
output_image_data = None
intput_image_data = None


input_height_data = gdal.Open(height_filepath);
srcband = input_height_data.GetRasterBand(1);
height_nodata = srcband.GetNoDataValue()
print('warping dem to long lat projection...please wait...')
output_height_filepath = output_path + layer_name + '_dem.tif'
output_height_data = gdal.Warp(output_height_filepath, input_height_data, dstSRS='+proj=longlat')
output_height_data = None
input_height_data = None

#create .vrt files required for alpha channel
image_vrt_filepath = output_path + layer_name + ".vrt"
height_vrt_filepath = output_path + layer_name + "_dem.vrt"

vrt_options = gdal.BuildVRTOptions(outputBounds=(-180, -90, 180, 90), addAlpha=True)
image_vrt = gdal.BuildVRT(image_vrt_filepath, output_image_filename, options=vrt_options)
image_vrt = None

vrt_options = gdal.BuildVRTOptions(outputBounds=(-180, -90, 180, 90), addAlpha=True)
height_vrt = gdal.BuildVRT(height_vrt_filepath, output_height_filepath, options=vrt_options)
height_vrt = None

#hack to add nodata value to alpha channel
wrote_first = False
with open(image_vrt_filepath, "r") as in_file:
    buf = in_file.readlines()
with open(image_vrt_filepath, "w") as out_file:
    for line in buf:
        if line.startswith("    <ColorInterp>Alpha</ColorInterp>"):
            line = f"<NoDataValue>{image_nodata}</NoDataValue>\n"+ line
            wrote_first = True
        if line.startswith("    </ComplexSource>") and wrote_first :
            line = f"<NODATA>{image_nodata}</NODATA>\n"+ line
        out_file.write(line)
#repeat for height
wrote_first = False
with open(height_vrt_filepath, "r") as in_file:
    buf = in_file.readlines()
with open(height_vrt_filepath, "w") as out_file:
    for line in buf:
        if line.startswith("    <ColorInterp>Alpha</ColorInterp>"):
            line = f"<NoDataValue>{height_nodata}</NoDataValue>\n"+ line
            wrote_first = True
        if line.startswith("    </ComplexSource>") and wrote_first :
            line = f"<NODATA>{height_nodata}</NODATA>\n"+ line
        out_file.write(line)        

warping image to long lat projection...please wait...
warping dem to long lat projection...please wait...


In [22]:
#create info file for openspace
f = open(output_path + layer_name + ".info", "a")
f.write("Name=\"" + layer_name + "\"\n");
f.write("Identifier=\"" + layer_name + "\"\n");
f.write("Description=\"" + layer_description + "\"\n");
f.write("ColorFile=\"" + layer_name + ".vrt\"\n");
f.write("HeightFile=\"" + layer_name + "_dem.vrt\"\n");
f.close()

In [23]:
#send command to add layers
#connect to openspace and pause time
from websocket import create_connection
import json
url = "localhost:4682"
ws_endpoint = f"ws://localhost:4682/websocket"
ws = create_connection(ws_endpoint)
#add layers to globe
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.globebrowsing.addBlendingLayersFromDirectory",
                              "arguments": [output_path, globe_for_layers],
                              "return": False}})
ws.send(message)
#refresh menu
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.setPropertyValueSingle",
                              "arguments": ['Modules.CefWebGui.Reload', None],
                              "return": False}})
ws.send(message)
ws.close()

In [ ]:
#future cell could find center point and navigate globe with openspace.globebrowsing.gotogeo and enable the layer